In [3]:
import sys
sys.path.append('..')
sys.path.append('../Libraries/')
from laspy.file import File
import pclpy
import ARUSTUFF
import Plane
import Corners
import cv2
import imp
import glob
import os
import numpy as np
import pcl
import open3d
#import pdal
from Plane import tic,toc

In [12]:
for i in range(1,7):
    #pc = pclpy.io.las.read(r'../..\EuroSDR_DataRelease\EuroSDR_DataRelease/TLS_Benchmarking_Plot_'+str(i)+'_MS.las', "PointXYZ")
    #pclpy.pcl.io.savePCDFile('Clouds/TLS_Benchmarking_Plot_'+str(i)+'_MS.pcd',pc, binary_mode = True)
    print('start',i)
    tic()
    mm = pclpy.pcl.PointCloud.PointXYZ()
    pclpy.pcl.io.loadPCDFile('Clouds/TLS_Benchmarking_Plot_'+str(i)+'_MS.pcd',mm)
    toc()

    len(mm.xyz)

    mm.xyz.shape

    x2,y2,z2 = np.max(mm.xyz,axis=0)
    x1,y1,z1 = np.min(mm.xyz,axis=0)
    x2,y2,z2,x1,y1,z1

    x2,y2,z2,x1,y1,z1

    ptfilter = pclpy.pcl.filters.CropBox.PointXYZ()
    ptfilter.setInputCloud(mm)
    ptfilter.setMin([x1,y1,z1,0])
    ptfilter.setMax([(x2-x1)/2+x1-0.000001,(y2-y1)/2+y1-0.000001,z2,0])
    Cropedmm11 = pclpy.pcl.PointCloud.PointXYZ()
    ptfilter.filter(Cropedmm11)

    ptfilter.setMin([(x2-x1)/2+x1,y1,z1,0])
    ptfilter.setMax([x2,(y2-y1)/2+y1-0.000001,z2,0])
    Cropedmm21 = pclpy.pcl.PointCloud.PointXYZ()
    ptfilter.filter(Cropedmm21)

    ptfilter.setMin([(x2-x1)/2+x1,(y2-y1)/2+y1,z1,0])
    ptfilter.setMax([x2,y2,z2,0])
    Cropedmm22 = pclpy.pcl.PointCloud.PointXYZ()
    ptfilter.filter(Cropedmm22)

    ptfilter.setMin([x1,(y2-y1)/2+y1,z1,0])
    ptfilter.setMax([(x2-x1)/2+x1-0.000001,y2,z2,0])
    Cropedmm12 = pclpy.pcl.PointCloud.PointXYZ()
    ptfilter.filter(Cropedmm12)

    Cropedmm11.xyz.shape[0]+Cropedmm21.xyz.shape[0]+Cropedmm22.xyz.shape[0]+Cropedmm12.xyz.shape[0]

    LS = 0.03

    
    VF = pclpy.pcl.filters.VoxelGrid.PointXYZ()
    VF.setLeafSize(LS,LS,LS)
    VF.setInputCloud(Cropedmm11)
    VFCropedmm11 = pclpy.pcl.PointCloud.PointXYZ()
    VF.filter(VFCropedmm11)
    Cropedmm11.xyz.shape,VFCropedmm11.xyz.shape

    VF.setLeafSize(LS,LS,LS)
    VF.setInputCloud(Cropedmm12)
    VFCropedmm12 = pclpy.pcl.PointCloud.PointXYZ()
    VF.filter(VFCropedmm12)
    Cropedmm12.xyz.shape,VFCropedmm12.xyz.shape

    VF.setLeafSize(LS,LS,LS)
    VF.setInputCloud(Cropedmm21)
    VFCropedmm21 = pclpy.pcl.PointCloud.PointXYZ()
    VF.filter(VFCropedmm21)
    Cropedmm21.xyz.shape,VFCropedmm21.xyz.shape

    VF.setLeafSize(LS,LS,LS)
    VF.setInputCloud(Cropedmm22)
    VFCropedmm22 = pclpy.pcl.PointCloud.PointXYZ()
    VF.filter(VFCropedmm22)
    Cropedmm22.xyz.shape,VFCropedmm22.xyz.shape

    FINALVF = pclpy.pcl.PointCloud.PointXYZ(np.concatenate([VFCropedmm11.xyz, VFCropedmm12.xyz, VFCropedmm21.xyz, VFCropedmm22.xyz]))
    FINALVF.xyz.shape,mm.xyz.shape
    pclpy.pcl.io.savePCDFile('downsampledlesscloudEURO'+str(i)+'.pcd',FINALVF, binary_mode = True)

    tic()
    mmO = pclpy.pcl.PointCloud.PointXYZRGB()
    pclpy.pcl.io.loadPCDFile('downsampledlesscloudEURO'+str(i)+'.pcd',mmO)
    toc()
    print(mm.xyz.shape,mmO.xyz.shape)

start 1
(111065193, 3) (9360346, 3)
start 2
(113724091, 3) (9230594, 3)
start 3
(119939391, 3) (11080140, 3)
start 4
(129151322, 3) (11288571, 3)
start 5
(124507186, 3) (8731507, 3)
start 6
(111068576, 3) (7745064, 3)


In [36]:
tic()
mm = pclpy.pcl.PointCloud.PointXYZRGB()
pclpy.pcl.io.loadPCDFile('../../Data/NistClouds/fullCloudSFM.pcd',mm)
toc()

7.35162339999988

In [37]:
Corners.PCL3dpaint(mm)

In [26]:
coef = np.array([[1.3510477, -2.97978098, -0.03949538,
        -15.06379948],
       [2.97252688, 1.34444401, 0.2500775,
        21.94287987],
       [-0.21151492, -0.1391405, 3.26219034,
        72.96143083],
       [0.0, 0.0, 0.0, 1.0]])

In [27]:
homogean = np.hstack([mm.xyz,np.ones_like(mm.xyz[:,0:1])])

In [32]:
position1 = np.array([100,0,0,0,0,1,0.2])
position2 = np.array([0,100,0,0,0,1,0.2])
position3 = np.array([100,100,0,0,0,1,0.2])
cil1 = Corners.makecylinder(model=position1,length=3,dense=100)
cil2 = Corners.makecylinder(model=position2,length=3,dense=100)
cil3 = Corners.makecylinder(model=position3,length=3,dense=100)

In [38]:
mmTrans = pclpy.pcl.PointCloud.PointXYZRGB()
pclpy.pcl.common.transformPointCloud(mm,mmTrans,coef)

In [53]:
Corners.PCL3dpaint([mmTrans,cil1,cil2,cil3])

In [61]:
len(mm.xyz)

mm.xyz.shape

x2,y2,z2 = np.max(mmTrans.xyz,axis=0)
x1,y1,z1 = np.min(mmTrans.xyz,axis=0)
x2,y2,z2,x1,y1,z1

x2,y2,z2,x1,y1,z1

ptfilter = pclpy.pcl.filters.CropBox.PointXYZRGB()
ptfilter.setInputCloud(mmTrans)
ptfilter.setMin([x1,y1,z1,0])
ptfilter.setMax([(x2-x1)/2+x1-0.000001,(y2-y1)/2+y1-0.000001,z2,0])
Cropedmm11 = pclpy.pcl.PointCloud.PointXYZRGB()
ptfilter.filter(Cropedmm11)

ptfilter.setMin([(x2-x1)/2+x1,y1,z1,0])
ptfilter.setMax([x2,(y2-y1)/2+y1-0.000001,z2,0])
Cropedmm21 = pclpy.pcl.PointCloud.PointXYZRGB()
ptfilter.filter(Cropedmm21)

ptfilter.setMin([(x2-x1)/2+x1,(y2-y1)/2+y1,z1,0])
ptfilter.setMax([x2,y2,z2,0])
Cropedmm22 = pclpy.pcl.PointCloud.PointXYZRGB()
ptfilter.filter(Cropedmm22)

ptfilter.setMin([x1,(y2-y1)/2+y1,z1,0])
ptfilter.setMax([(x2-x1)/2+x1-0.000001,y2,z2,0])
Cropedmm12 = pclpy.pcl.PointCloud.PointXYZRGB()
ptfilter.filter(Cropedmm12)

Cropedmm11.xyz.shape[0]+Cropedmm21.xyz.shape[0]+Cropedmm22.xyz.shape[0]+Cropedmm12.xyz.shape[0]

LS = 0.10


VF = pclpy.pcl.filters.VoxelGrid.PointXYZRGB()
VF.setLeafSize(LS,LS,LS)
VF.setInputCloud(Cropedmm11)
VFCropedmm11 = pclpy.pcl.PointCloud.PointXYZRGB()
VF.filter(VFCropedmm11)
Cropedmm11.xyz.shape,VFCropedmm11.xyz.shape

VF.setLeafSize(LS,LS,LS)
VF.setInputCloud(Cropedmm12)
VFCropedmm12 = pclpy.pcl.PointCloud.PointXYZRGB()
VF.filter(VFCropedmm12)
Cropedmm12.xyz.shape,VFCropedmm12.xyz.shape

VF.setLeafSize(LS,LS,LS)
VF.setInputCloud(Cropedmm21)
VFCropedmm21 = pclpy.pcl.PointCloud.PointXYZRGB()
VF.filter(VFCropedmm21)
Cropedmm21.xyz.shape,VFCropedmm21.xyz.shape

VF.setLeafSize(LS,LS,LS)
VF.setInputCloud(Cropedmm22)
VFCropedmm22 = pclpy.pcl.PointCloud.PointXYZRGB()
VF.filter(VFCropedmm22)
Cropedmm22.xyz.shape,VFCropedmm22.xyz.shape

FINALVF = pclpy.pcl.PointCloud.PointXYZRGB(np.vstack([VFCropedmm11.xyz, VFCropedmm12.xyz, VFCropedmm21.xyz, VFCropedmm22.xyz]),
                                                     np.vstack([VFCropedmm11.rgb, VFCropedmm12.rgb, VFCropedmm21.rgb, VFCropedmm22.rgb]))
FINALVF.xyz.shape,mm.xyz.shape
pclpy.pcl.io.savePCDFile('downsampledNistBetter.pcd',FINALVF, binary_mode = True)

tic()
mmO = pclpy.pcl.PointCloud.PointXYZRGB()
pclpy.pcl.io.loadPCDFile('downsampledNistBetter.pcd',mmO)
toc()
print(mm.xyz.shape,mmO.xyz.shape)

(67009720, 3) (14025247, 3)
